# Logon Scripts (UserInitMprLogonScript)
Detects creation or execution of UserInitMprLogonScript persistence method

## Rule Content
```
- action: global
  title: Logon Scripts (UserInitMprLogonScript)
  id: 0a98a10c-685d-4ab0-bddc-b6bdd1d48458
  status: experimental
  description: Detects creation or execution of UserInitMprLogonScript persistence
    method
  references:
  - https://attack.mitre.org/techniques/T1037/
  tags:
  - attack.t1037
  - attack.persistence
  - attack.lateral_movement
  author: Tom Ueltschi (@c_APT_ure)
  falsepositives:
  - exclude legitimate logon scripts
  - penetration tests, red teaming
  level: high
- logsource:
    category: process_creation
    product: windows
  detection:
    exec_selection:
      ParentImage: '*\userinit.exe'
    exec_exclusion1:
      Image: '*\explorer.exe'
    exec_exclusion2:
      CommandLine: '*\netlogon.bat'
    condition: exec_selection and not exec_exclusion1 and not exec_exclusion2
- logsource:
    category: process_creation
    product: windows
  detection:
    create_keywords_cli:
      CommandLine: '*UserInitMprLogonScript*'
    condition: create_keywords_cli
- logsource:
    product: windows
    service: sysmon
  detection:
    create_selection_reg:
      EventID:
      - 11
      - 12
      - 13
      - 14
    create_keywords_reg:
      TargetObject: '*UserInitMprLogonScript*'
    condition: create_selection_reg and create_keywords_reg

```

## Querying Elasticsearch

### Import Libraries

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import pandas as pd

### Initialize Elasticsearch client

In [ ]:
es = Elasticsearch(['http://helk-elasticsearch:9200'])
searchContext = Search(using=es, index='logs-*', doc_type='doc')

### Run Elasticsearch Query

In [ ]:
s = searchContext.query('query_string', query='((process_parent_path.keyword:*\\userinit.exe AND (NOT (process_path.keyword:*\\explorer.exe))) AND (NOT (process_command_line.keyword:*\\netlogon.bat)))')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

In [ ]:
s = searchContext.query('query_string', query='process_command_line.keyword:*UserInitMprLogonScript*')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

In [ ]:
s = searchContext.query('query_string', query='(event_id:("11" OR "12" OR "13" OR "14") AND registry_key_path.keyword:*UserInitMprLogonScript*)')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

### Show Results

In [ ]:
df.head()